전국문화축제표준데이터

https://www.data.go.kr/data/15013104/standard.do

- api key 발급
- url 만들기
- requests 요청
- parsing


#### 1페이지 데이터 수집


In [36]:
key_decoding = "/2lArsAtATWhH3o+FiNjJGEcuSMp9EYpJMfKIMnMmm31xL5n9e9/NKsPipzZMV5lsP9N1z00HiYszRTw8IjEwA=="

import pandas as pd
import requests
import json

# ------------------------------------------------
# url 생성 및 파라미터 정의
#   - opneAPI 활용 가이드에 따름
#   - 키 발급
#   - 요청 url 생성
#   - 파라미터 정의
#   - json 타입으로 응답받도록 지정
# ------------------------------------------------
url = "http://apis.data.go.kr/1140100/minAnalsInfoView7/minPotholeLatLonInfo"

params = {"serviceKey": key_decoding, "dateFrom": "20250101", "dateTo": "20250115"}

# -----------------------------------
# 요청 및 응답
#   응답객체는 json 타입
# -----------------------------------
response = requests.get(url, params=params)
print(response.text)

# -----------------------------------
# parsing
#   json --> dict
# -----------------------------------
data = response.json()
print(data)

# -----------------------------------
# 데이터 추출
# -----------------------------------
print(len(data["body"]["items"]))

{"body":{"items":[{"item":{"create_dt":"20250101","event_lat":"35.31881239929581","event_lon":"129.23959808199848"}},{"item":{"create_dt":"20250101","event_lat":"36.329056113174786","event_lon":"127.38723507032297"}},{"item":{"create_dt":"20250101","event_lat":"36.46705831666666","event_lon":"127.27526243333331"}},{"item":{"create_dt":"20250101","event_lat":"36.8396110534668","event_lon":"126.9688949584961"}},{"item":{"create_dt":"20250101","event_lat":"36.89813995361328","event_lon":"126.62808990478516"}},{"item":{"create_dt":"20250101","event_lat":"37.24326366576369","event_lon":"127.03756453582656"}},{"item":{"create_dt":"20250101","event_lat":"37.65962600708008","event_lon":"126.89498901367188"}},{"item":{"create_dt":"20250102","event_lat":"33.48398333333333","event_lon":"126.4269866666667"}},{"item":{"create_dt":"20250102","event_lat":"35.16037947035875","event_lon":"126.79756117065138"}},{"item":{"create_dt":"20250102","event_lat":"35.22125244140625","event_lon":"128.586853027343

#### 모든 페이지 데이터 수집


In [55]:
key_decoding = "/2lArsAtATWhH3o+FiNjJGEcuSMp9EYpJMfKIMnMmm31xL5n9e9/NKsPipzZMV5lsP9N1z00HiYszRTw8IjEwA=="

import pandas as pd
import requests
import json
import datetime
import time

pothole_data = []

url = "http://apis.data.go.kr/1140100/minAnalsInfoView7/minPotholeLatLonInfo"

start_date = datetime.datetime(2021, 1, 1)
end_date = start_date + datetime.timedelta(days=14)

# -----------------------------------
# 여러 페이지 데이터를 받기 위해 반복
# -----------------------------------
while start_date < datetime.datetime(2021, 12, 31):
    time.sleep(1)
    params = {
        "serviceKey": key_decoding,
        "dateFrom": start_date.strftime("%Y%m%d"),
        "dateTo": end_date.strftime("%Y%m%d"),
    }

    print(params["dateFrom"], params["dateTo"])

    response = requests.get(url, params=params)
    print(response.text)

    # -----------------------------------
    # parsing
    #   json --> dict
    # -----------------------------------
    data = response.json()["body"]["items"]

    data = list(map(
        lambda data: {
            "date": data["item"]["create_dt"],
            "latitude": data["item"]["event_lat"],
            "longitude": data["item"]["event_lon"],
        },
        data,
    ))
    pothole_data.extend(data)

    start_date = start_date + datetime.timedelta(days=15)
    end_date = start_date + datetime.timedelta(days=14)

# -------------------------------
# 파일로 저장
# -------------------------------
df = pd.DataFrame(pothole_data)
prefix = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
df.to_csv(f"../data/pothole_{prefix}.csv", index=False, encoding="utf-8")

df

# 위 데이터프레임에 작성되어있는 date, latitude, longitude를 folium을 사용하여 지도에 마커를 시각화
import folium

# 'date' 컬럼을 datetime 객체로 변환 (문자열인 경우)
if pd.api.types.is_string_dtype(df['date']):
    df['date_dt'] = pd.to_datetime(df['date'])
else:
    df['date_dt'] = df['date'] # 이미 datetime 객체인 경우

df['year_month'] = df['date_dt'].dt.strftime('%Y-%m')

# 연도-월별 색상 매핑
# 사용 가능한 Folium 색상 리스트
available_colors = [
    'red', 'blue', 'green', 'purple', 'orange', 'darkred',
    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
    'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
    'gray', 'black', 'lightgray'
]

unique_year_months = sorted(df['year_month'].unique())
color_map = {
    year_month: available_colors[i % len(available_colors)]
    for i, year_month in enumerate(unique_year_months)
}

# Folium 지도 생성
if not df.empty:
    map_center = [df['latitude'].iloc[0], df['longitude'].iloc[0]]
    m = folium.Map(location=map_center, zoom_start=7)
else:
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=7)

# DataFrame의 각 행에 대해 마커 추가
for idx, row in df.iterrows():
    year_month_val = row['year_month']
    marker_color = color_map.get(year_month_val, 'gray') # 매핑에 없는 경우 기본색 'gray'

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"이름: {row['name']}<br>날짜: {row['date_dt'].strftime('%Y-%m-%d')}<br>위도: {row['latitude']:.4f}<br>경도: {row['longitude']:.4f}",
        tooltip=f"{row['name']} ({year_month_val})",
        icon=folium.Icon(color=marker_color, icon='info-sign') # icon 모양도 지정 가능
    ).add_to(m)

# 범례 추가
# HTML로 범례를 만들어 지도에 추가할 수 있습니다.
legend_html = """
     <div style="position: fixed;
                 bottom: 50px; left: 50px; width: 150px; height: auto;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity:0.9;">
       &nbsp; <b>연도-월별 색상</b> <br>
"""
for year_month, color in color_map.items():
    legend_html += f'&nbsp; <i class="fa fa-map-marker fa-2x" style="color:{color}"></i> &nbsp; {year_month} <br>'
legend_html += "</div>"
m.get_root().html.add_child(folium.Element(legend_html))


# 7. 지도 저장 또는 표시
m.save("map_with_colored_markers.html")
print("map_with_colored_markers.html 파일이 생성되었습니다. 웹 브라우저에서 열어보세요.")

20210101 20210115
<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>LIMITED_NUMBER_OF_SERVICE_REQUESTS_EXCEEDS_ERROR</returnAuthMsg>
		<returnReasonCode>22</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)